# Policy Generator

+ Tag the traffic trace with the version
+ Ability to compare real vs prediction
+ Ability to generate predictions with variable accuracies
+ Group time periods based on Policy settings
+ Tag each group by decided version 


### Tag Traffic

In [190]:
import requests
import yaml
r = requests.get('https://raw.githubusercontent.com/CN-UPB/Pishahang/mvp-thesis/pish-examples/pwm-scripts/descriptors/multiversion/cirros1_mv_vnfd.yml')
# print(r.text)
VNFD = yaml.load(r.text, Loader=yaml.FullLoader)

VNFD["versions"]

# for _vm_type_key, _vm_type_value in VNFD["versions"].items():
#     print(_vm_type_key)    
#     for _vm_version_key, _vm_version_value in _vm_type_value.items():
#         print(_vm_version_key)
#         print(_vm_version_value)

{'virtual_deployment_units_vm': {'cirros-image-1-vm': {'cost_per_min': '2',
   'max_data_rate': 1200,
   'management_overhead': 60},
  'cirros-image-2-vm': {'cost_per_min': '2',
   'max_data_rate': 1200,
   'management_overhead': 60}},
 'virtual_deployment_units_gpu': {'cirros-image-1-gpu': {'cost_per_min': '5',
   'max_data_rate': 3000,
   'management_overhead': 6},
  'cirros-image-2-gpu': {'cost_per_min': '5',
   'max_data_rate': 3000,
   'management_overhead': 6}},
 'virtual_deployment_units_con': {'cirros-image-1-con': {'cost_per_min': '1',
   'max_data_rate': 600,
   'management_overhead': 4},
  'cirros-image-2-con': {'cost_per_min': '1',
   'max_data_rate': 600,
   'management_overhead': 4}}}

In [217]:
import pandas as pd
import numpy as np

LOOK_AHEAD = 15 # Mins

traffic_training_complete = pd.read_csv(r'/plugins/son-mano-traffic-forecast/notebooks/data/dataset_7_day_traffic.csv', index_col=0)
print(traffic_training_complete.shape)
traffic_training_complete.head(5)

(22188, 2)


,sent,received
1583107200,184.899601,0.287412
1583107260,133.495889,0.430124
1583107320,231.145392,0.632607
1583107380,189.214077,0.662958
1583107440,178.089039,0.665811


In [218]:
traffic_grouped = traffic_training_complete.groupby(
                    np.arange(len(traffic_training_complete))//LOOK_AHEAD).agg(['mean', 'std', 'min', 'max'])
# result.index = df.loc[1::2, 'Idx']
print(traffic_grouped.shape)
traffic_grouped.head(22)

(1480, 8)


sent                                     received            \
          mean        std         min         max      mean       std   
0   210.761322  54.355541  116.621570  295.154365  0.610675  0.200633   
1   202.214353  51.653250  101.729590  279.378450  0.532455  0.270002   
2   217.926026  66.810697  106.225388  295.278818  0.338812  0.284006   
3   191.597688  70.571346  114.998696  292.908857  0.490285  0.344671   
4   202.095334  61.568794  100.540172  286.236149  0.492254  0.280906   
5   194.107488  62.969057  103.581750  293.625565  0.541595  0.310078   
6   204.793832  49.584700  113.997633  283.576267  0.555603  0.257493   
7   177.424974  60.289400  104.076365  290.608109  0.515565  0.262725   
8   183.351098  47.777718  100.239187  270.704376  0.536273  0.302871   
9   206.130413  67.420513  101.780853  293.655324  0.428524  0.267145   
10  208.737601  56.160780  110.942039  286.955559  0.416499  0.328815   
11  185.711121  46.599092  129.558386  297.663908  0.502506  0.322446   
12  222.827917  63.367448  101.613815  287.331771  0.502947  0.333768   
13  202.528601  47.719461  128.653978  296.320646  0.421220  0.311054   
14  202.719455  57.925459  104.401682  290.714593  0.601917  0.334463   
15  210.837275  50.564810  122.047204  299.293981  0.547538  0.289013   
16  234.208394  66.427920  101.499823  299.505989  0.517931  0.256588   
17  196.547458  66.254967  101.285512  294.888127  0.529361  0.302235   
18  202.419747  63.699977  116.720248  298.585114  0.485297  0.324084   
19  243.187622  41.303316  143.978900  297.688550  0.506457  0.371980   
20  202.210424  53.584031  128.060077  279.754289  0.491285  0.226424   
21  177.292844  60.911360  100.049863  296.425575  0.569569  0.266215   

                        
         min       max  
0   0.235468  0.901092  
1   0.063792  0.988732  
2   0.021534  0.940878  
3   0.000342  0.962522  
4   0.011211  0.981805  
5   0.030187  0.983977  
6   0.098460  0.905997  
7   0.096960  0.925005  
8   0.074150  0.985523  
9   0.071989  0.988942  
10  0.005214  0.837424  
11  0.039945  0.962812  
12  0.030005  0.983264  
13  0.014829  0.989440  
14  0.074912  0.972599  
15  0.043848  0.960434  
16  0.128174  0.946564  
17  0.085989  0.925857  
18  0.031171  0.974594  
19  0.023620  0.980983  
20  0.230262  0.941790  
21  0.150094  0.893850

# Policy Decision Steps

1. Get all supported versions for current datarate prediction
    - If nothing found, return the maximum

2. Give score to all the supported versions based on the policy configurations
    - Cost (-)
    - Support deviation (+)
    - Over Provision (-)
    - Same Version (+)
    - Overhead (-)

----
# Decision Matrix    

|                    	| Weights 	| Version1 	| Version2 	| Version3 	|
|--------------------	|:-------:	|:--------:	|:--------:	|:--------:	|
| Cost (-)           	|    4    	|          	|          	|          	|
| Support deviation (+) |    3    	|          	|          	|          	|
| Over Provision (-) 	|    -3   	|          	|          	|          	|
| Same Version (+)   	|    3    	|          	|          	|          	|
| Overhead (-)       	|    -4   	|          	|          	|          	|

In [219]:
def find_max_datarate_version(versions):
    _max_datarate = 0

    for _vm_type_key, _vm_type_value in versions.items():
        # print(_vm_type_key)    

        for _vm_version_key, _vm_version_value in _vm_type_value.items():
            # print(_vm_version_key)
            # print(_vm_version_value["max_data_rate"])

            if _vm_version_value["max_data_rate"] > _max_datarate:
                _max_datarate = _vm_version_value["max_data_rate"]
                _max_datarate_version = { _vm_type_key: { _vm_version_key : _vm_version_value } }

    return _max_datarate_version

def get_supported_versions(prediction, versions):
    # Iterate versions
    datarate_supported_versions = {}

    for _vm_type_key, _vm_type_value in versions.items():
        # print(_vm_type_key)    

        for _vm_version_key, _vm_version_value in _vm_type_value.items():
            # print(_vm_version_key)
            # print(_vm_version_value["max_data_rate"])
            # print(prediction["mean"])
            if _vm_version_value["max_data_rate"] >= prediction["mean"]:
                # check if key present else add
                if _vm_type_key in datarate_supported_versions:
                    datarate_supported_versions[_vm_type_key][_vm_version_key] = _vm_version_value
                else:
                    datarate_supported_versions[_vm_type_key] = {}
                    datarate_supported_versions[_vm_type_key][_vm_version_key] = _vm_version_value

    if len(datarate_supported_versions) == 0:
        return find_max_datarate_version(versions)
    return datarate_supported_versions


In [220]:
import numpy as np
from sklearn import preprocessing

_SCORE_MIN, _SCORE_MAX = 1, 5


def interpolate_array(values, min=_SCORE_MIN, max=_SCORE_MAX):
    return np.interp(values, (values.min(), values.max()), (min, max))

def build_decision_matrix(prediction, meta, versions):
    _decision_matrix = {}
    for _vm_type_key, _vm_type_value in versions.items():
        for _vm_version_key, _vm_version_value in _vm_type_value.items():
                if _vm_type_key not in _decision_matrix:
                    _decision_matrix[_vm_type_key] = {}
                if _vm_version_key not in _decision_matrix[_vm_type_key]:
                    _decision_matrix[_vm_type_key][_vm_version_key] = {}

                # Cost
                _decision_matrix[_vm_type_key][_vm_version_key]["cost"] = _vm_version_value['cost_per_min']

                # Support deviation
                if _vm_version_value['max_data_rate'] > (prediction['mean'] + prediction['std']):
                    _decision_matrix[_vm_type_key][_vm_version_key]["support_deviation"] = 5
                else:
                    _decision_matrix[_vm_type_key][_vm_version_key]["support_deviation"] = 1

                # Over Provision
                _decision_matrix[_vm_type_key][_vm_version_key]["over_provision"] = int(_vm_version_value['max_data_rate']) - int(prediction['mean'])

                # Same Version
                if meta["current_version"] == _vm_version_key:
                    _decision_matrix[_vm_type_key][_vm_version_key]["same_version"] = 5
                else:
                    _decision_matrix[_vm_type_key][_vm_version_key]["same_version"] = 1

                # Overhead
                _decision_matrix[_vm_type_key][_vm_version_key]["overhead"] = _vm_version_value['management_overhead']


    decision_matrix_df = pd.DataFrame.from_dict({(i,j): _decision_matrix[i][j] 
                                for i in _decision_matrix.keys() 
                                for j in _decision_matrix[i].keys()},
                                orient='index')

    decision_matrix_df["over_provision"] = interpolate_array(decision_matrix_df["over_provision"])
    decision_matrix_df["cost"] = interpolate_array(decision_matrix_df["cost"])
    decision_matrix_df["overhead"] = interpolate_array(decision_matrix_df["overhead"])

    return decision_matrix_df

row_series = { "mean": 800, "std": 100, "min": 800, "max": 1800 }
meta = { "current_version": "cirros-image-1-gpu" }
supported_versions = get_supported_versions(prediction=row_series, versions=VNFD["versions"])
decision_matrix_df = build_decision_matrix(prediction=row_series, meta=meta, versions=supported_versions)

decision_matrix_df

cost  support_deviation  \
virtual_deployment_units_vm  cirros-image-1-vm    1.0                  5   
                             cirros-image-2-vm    1.0                  5   
virtual_deployment_units_gpu cirros-image-1-gpu   5.0                  5   
                             cirros-image-2-gpu   5.0                  5   

                                                 over_provision  same_version  \
virtual_deployment_units_vm  cirros-image-1-vm              1.0             1   
                             cirros-image-2-vm              1.0             1   
virtual_deployment_units_gpu cirros-image-1-gpu             5.0             5   
                             cirros-image-2-gpu             5.0             1   

                                                 overhead  
virtual_deployment_units_vm  cirros-image-1-vm        5.0  
                             cirros-image-2-vm        5.0  
virtual_deployment_units_gpu cirros-image-1-gpu       1.0  
                             cirros-image-2-gpu       1.0

In [221]:
# cost, support_deviation, over_provision, same_version, overhead
WEIGHTS = [-4, 2, -4, 3, -3]

def get_policy_decision(decision_matrix, weights):

    for index_label, row_series in decision_matrix.iterrows():
        _row = np.array([row_series.cost, row_series.support_deviation, row_series.over_provision, row_series.same_version, row_series.overhead])

        decision_matrix.at[index_label , 'score'] = np.dot(np.array(weights), _row)

    _version = decision_matrix[decision_matrix.score == decision_matrix.score.max()].index[0]
    # print(decision_matrix)
    # print(":".join(_version))
    return _version

get_policy_decision(decision_matrix_df, WEIGHTS)

('virtual_deployment_units_vm', 'cirros-image-1-vm')

In [222]:
WEIGHTS = [-4, 2, -4, 3, -3]


traffic_grouped_sent = traffic_grouped['sent']
# traffic_grouped_sent.plot()

# iterate over the dataframe row by row and set version
for index_label, row_series in traffic_grouped_sent.iterrows():
   meta = { "current_version": "cirros-image-1-con" }
   supported_versions = get_supported_versions(prediction=row_series, versions=VNFD["versions"])
   decision_matrix_df = build_decision_matrix(prediction=row_series, meta=meta, versions=supported_versions)
   traffic_grouped_sent.at[index_label , 'version'] = ":".join(get_policy_decision(decision_matrix_df, WEIGHTS))
   # print(row_series)
   # print(decision_matrix_df)
   # print("\n\n")

   

traffic_grouped_sent.head()

,mean,std,min,max,version
0,210.761322,54.355541,116.621570,295.154365,virtual_deployment_units_con:cirros-image-1-con
1,202.214353,51.653250,101.729590,279.378450,virtual_deployment_units_con:cirros-image-1-con
2,217.926026,66.810697,106.225388,295.278818,virtual_deployment_units_con:cirros-image-1-con
3,191.597688,70.571346,114.998696,292.908857,virtual_deployment_units_con:cirros-image-1-con
4,202.095334,61.568794,100.540172,286.236149,virtual_deployment_units_con:cirros-image-1-con
